In [1]:
import numpy as np
from network import *
from utils import *
import tensorflow as tf
import os 
import pickle
import subprocess
import glob
from pathlib import Path
from music21 import converter, instrument, note, chord, stream
from tqdm import tqdm

In [2]:
notes = []
midi_dir = Path('./adl_piano_midi')
midi_file_paths = list(midi_dir.rglob('*.mid'))

sample = np.random.choice(midi_file_paths, 200)

# midi_file_paths
for midi in tqdm(sample):
    stream = converter.parse(midi)
    instrument_ = instrument.partitionByInstrument(stream)
    parse_ = instrument_.parts[0].recurse() if instrument_ else stream.flat.notes
    for parse in parse_:
        if isinstance(parse, chord.Chord):
            for i in parse.normalOrder:
                notes.append('.'.join(str(i)))
        elif isinstance(parse, note.Note):
            notes.append(str(parse.pitch))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [05:52<00:00,  1.76s/it]


In [3]:
pitch_num = len(set(notes))
pitch_num, len(notes)

(96, 148381)

In [4]:
notes

['A5',
 'B-5',
 'G5',
 'A5',
 '5',
 '9',
 '0',
 'G5',
 'C4',
 'C6',
 'C4',
 'A5',
 'B-5',
 'G5',
 'A5',
 '5',
 '9',
 '0',
 'G5',
 'C6',
 'B-5',
 'G5',
 '5',
 '9',
 '0',
 'A5',
 'B-5',
 'G5',
 'A5',
 '5',
 '9',
 '0',
 'G5',
 'C4',
 'C6',
 'C4',
 'A5',
 'B-5',
 'G5',
 'A5',
 '5',
 '9',
 '0',
 'G5',
 'C6',
 'B-5',
 'G5',
 '5',
 '9',
 '0',
 'G5',
 'G3',
 '9',
 '0',
 '4',
 '7',
 'C6',
 'G3',
 'G5',
 'G3',
 'B5',
 'G3',
 'G5',
 'G3',
 'C6',
 'G3',
 'G5',
 'G3',
 'G5',
 'G3',
 '9',
 '0',
 '4',
 '7',
 'C6',
 'G3',
 'G5',
 'G3',
 '1.1',
 '2',
 '5',
 '7',
 '2',
 '5',
 '9',
 '7',
 '1.1',
 'C6',
 'C4',
 'B3',
 'C4',
 'A5',
 'B-5',
 'G5',
 'A5',
 '5',
 '9',
 '0',
 'G5',
 'C4',
 'C6',
 'C4',
 'C6',
 'B-5',
 'G5',
 'A5',
 '5',
 '9',
 '0',
 'G5',
 'C6',
 'B-5',
 'G5',
 '5',
 '9',
 '0',
 'G5',
 'G3',
 '9',
 '0',
 '4',
 '7',
 'C6',
 'G3',
 'G5',
 'G3',
 'B5',
 'G3',
 'G5',
 'G3',
 'C6',
 'G3',
 'G5',
 'G3',
 'G5',
 'G3',
 '9',
 '0',
 '4',
 '7',
 'C6',
 'G3',
 'G5',
 'G3',
 '1.1',
 '2',
 '5',
 '7',
 '2',

In [5]:
if not os.path.exists('data'):
    os.mkdir('data')

with open('data/notes', 'wb') as path:
    pickle.dump(notes, path)

In [6]:
seq_len = 100
pitch_types = sorted(set(pitch for pitch in notes))
pitch_int_dict  = dict((pitch, num) for num, pitch in enumerate(pitch_types))
int_pitch_dict  = dict((num, pitch) for num, pitch in enumerate(pitch_types))

input_seq = []
true_pitches = []
for i in tqdm(range(0, len(notes)-seq_len, 1)):
    # use a sliding window with length 100 and append all the windows to the input_seq
    input_seq.append([pitch_int_dict[pitch] for pitch in notes[i: i+seq_len]])
    true_pitches.append(pitch_int_dict[notes[i+seq_len]])
    
input_seq = np.reshape(input_seq,(len(input_seq),seq_len,1))

# Normalization
input_seq = input_seq/float(pitch_num)
true_pitches = tf.keras.utils.to_categorical(true_pitches)

# input_seq, true_pitches
input_seq.shape, true_pitches.shape

100%|███████████████████████████████████████████████████████████████████████| 148281/148281 [00:01<00:00, 80944.89it/s]


((148281, 100, 1), (148281, 96))

In [7]:
def neural_network(inputs, pitch_num, parameter_ = None):
    model = tf.keras.Sequential()
    
    # 1st layer
    model.add(tf.keras.layers.LSTM(512, input_shape=(inputs.shape[1], inputs.shape[2]), return_sequences=True))
    
    # 2nd and 3rd layers
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.LSTM(512,return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.LSTM(512))
    
    # Dense layer
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(pitch_num))
    
    # Activation layer
    model.add(tf.keras.layers.Activation('softmax'))
    
    # Compile the model
    optimizer=tf.keras.optimizers.RMSprop(0.0004)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    
    
    if parameter_ is not None:
        model.load_weights(parameter_)
    return model

In [10]:
model = neural_network(input_seq, pitch_num)

# filepath="weights-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint=tf.keras.callbacks.ModelCheckpoint(
#     filepath,
#     monitor='loss',
#     verbose=0,
#     save_best_only=True,
#     mode='min'
# )

# callbacks_list=[checkpoint]
    

# model.load_weights('./weights-30-2.1318.hdf5')
# model.load_weights('./weights3-05-0.0709.hdf5')
history = model.fit(input_seq, true_pitches, epochs=30, batch_size=128)

# model.save(filepath='./model',save_format='h5')

Epoch 1/30
148281/148281 [==============================] - 881s 6ms/sample - loss: 3.6785
Epoch 2/30
148281/148281 [==============================] - 855s 6ms/sample - loss: 3.4152
Epoch 3/30
148281/148281 [==============================] - 900s 6ms/sample - loss: 3.2227
Epoch 4/30
148281/148281 [==============================] - 863s 6ms/sample - loss: 3.0473
Epoch 5/30
148281/148281 [==============================] - 909s 6ms/sample - loss: 2.8607
Epoch 6/30
148281/148281 [==============================] - 883s 6ms/sample - loss: 2.6913
Epoch 7/30
148281/148281 [==============================] - 847s 6ms/sample - loss: 2.5364
Epoch 8/30
148281/148281 [==============================] - 886s 6ms/sample - loss: 2.3981
Epoch 9/30
148281/148281 [==============================] - 898s 6ms/sample - loss: 2.2693
Epoch 10/30
148281/148281 [==============================] - 874s 6ms/sample - loss: 2.1463
Epoch 11/30
148281/148281 [==============================] - 900s 6ms/sample - loss: 2.02

In [12]:
model.save('model.hdf5')

In [16]:
model.save('model.h5')

In [74]:
history = model.fit(input_seq, true_pitches, epochs=10, batch_size=128)

Epoch 1/10
148281/148281 [==============================] - 919s 6ms/sample - loss: 0.6875
Epoch 2/10
148281/148281 [==============================] - 943s 6ms/sample - loss: 0.6521
Epoch 3/10
148281/148281 [==============================] - 923s 6ms/sample - loss: 0.6256
Epoch 4/10
148281/148281 [==============================] - 906s 6ms/sample - loss: 0.5971
Epoch 5/10
148281/148281 [==============================] - 929s 6ms/sample - loss: 0.5688
Epoch 6/10
148281/148281 [==============================] - 912s 6ms/sample - loss: 0.5494
Epoch 7/10
148281/148281 [==============================] - 920s 6ms/sample - loss: 0.5243
Epoch 8/10
148281/148281 [==============================] - 884s 6ms/sample - loss: 0.5028
Epoch 9/10
148281/148281 [==============================] - 928s 6ms/sample - loss: 0.4846
Epoch 10/10
148281/148281 [==============================] - 906s 6ms/sample - loss: 0.4658


In [87]:
model.save('model1.hdf5')

In [89]:
history = model.fit(input_seq, true_pitches, epochs=5, batch_size=64)

Epoch 1/5
148281/148281 [==============================] - 1769s 12ms/sample - loss: 0.5740
Epoch 2/5
148281/148281 [==============================] - 1773s 12ms/sample - loss: 0.5676
Epoch 3/5
148281/148281 [==============================] - 1743s 12ms/sample - loss: 0.5568
Epoch 4/5
148281/148281 [==============================] - 1765s 12ms/sample - loss: 0.5383
Epoch 5/5
148281/148281 [==============================] - 1779s 12ms/sample - loss: 0.5227


In [105]:
model.save('model2.hdf5')

In [100]:
random_start_seq = []
random_start_seq = input_seq[np.random.randint(0, len(input_seq)-1)]

pred_gen = []

for pitch_idx in tqdm(range(550)):
    _input = np.reshape(random_start_seq, (1,len(random_start_seq), 1))
    _input = _input / float(pitch_num)
    pred = model.predict(_input, verbose=0)
    idx = np.argmax(pred)
    pred_gen.append(int_pitch_dict[idx])
    random_start_seq = random_start_seq[1:len(random_start_seq)]
    random_start_seq = np.append(random_start_seq, idx)


100%|████████████████████████████████████████████████████████████████████████████████| 550/550 [02:00<00:00,  4.56it/s]


In [101]:
pred_gen

['2',
 '6',
 '1.0',
 '2',
 '5',
 '3',
 '1',
 '1',
 '5',
 'B-4',
 '1.0',
 '2',
 '5',
 'B-4',
 '1.0',
 '2',
 '5',
 '3',
 '7',
 '1.0',
 '2',
 '5',
 '8',
 '1.0',
 '2',
 '5',
 '1.0',
 '2',
 '5',
 '1.0',
 '2',
 '5',
 '1.0',
 '2',
 '5',
 '8',
 '1.0',
 '2',
 '5',
 '8',
 '1.0',
 '1',
 '2',
 '3',
 '5',
 '1.0',
 '1.0',
 '2',
 '5',
 '9',
 '2',
 '5',
 '9',
 '1.0',
 '1',
 '2',
 '5',
 '1.0',
 '2',
 '5',
 '1.0',
 '4',
 '7',
 '1.0',
 '4',
 '7',
 '1.0',
 '4',
 '7',
 '1.0',
 '2',
 '5',
 '1.0',
 '2',
 '4',
 '6',
 '1.0',
 '1',
 '3',
 '6',
 '1.0',
 '0',
 '4',
 '7',
 '1.0',
 '3',
 '4',
 '8',
 '1.1',
 '4',
 '5',
 '1.0',
 '2',
 '2',
 '5',
 '9',
 '1',
 '4',
 '7',
 '1.0',
 '4',
 '9',
 '1.0',
 '4',
 '9',
 '1',
 '4',
 '9',
 '1',
 '4',
 '9',
 '1',
 '4',
 '4',
 '8',
 '1.1',
 '2',
 'F#3',
 'D4',
 '6',
 '1.1',
 '1',
 '4',
 '1',
 '4',
 '6',
 '9',
 '1',
 '9',
 '1',
 '3',
 'D4',
 'A4',
 '9',
 '0',
 '2',
 '9',
 '9',
 '0',
 '4',
 '2',
 '6',
 '7',
 '1.1',
 '2',
 '5',
 '9',
 '1.1',
 '2',
 '4',
 '9',
 '9',
 '1',
 '2',
 '4',
 

In [102]:
offset = 0
notes_gen = []

for num in tqdm(pred_gen):
    if(num.isdigit or '.' in num):
        note_chord = num.split('.')
        _notes = []
        for current_note in note_chord:
            if(current_note[0].isalpha):
                current_note = pitch_int_dict[current_note]
            _note = note.Note(int(current_note))
            _note.storedInstrument = instrument.Piano()
            _notes.append(_note)
            
        _chord = chord.Chord(_notes)
        _chord.offset = offset
        notes_gen.append(_chord)
    elif(num[0].isalpha):
        _note = note.Note(pitch_int_dict[num])
        _note.offset = offset
        _note.storedInstrument = instrument.Piano()
        notes_gen.append(_note)
    else:
        _note = note.Note(num)
        _note.offset = offset
        _note.storedInstrument = instrument.Piano()
        notes_gen.append(_note)
        
    offset = offset + 0.5


100%|██████████████████████████████████████████████████████████████████████████████| 550/550 [00:00<00:00, 8599.18it/s]


In [103]:
from music21 import stream

midi_s = stream.Stream(notes_gen)

midi_s.write('midi', fp='generation4.mid')

'generation4.mid'

In [104]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
__________